# Downloading and processing data

You need to download data in osm.pbf format:  
[Parts of the world](http://download.geofabrik.de/)  
[Cities](https://download.bbbike.org/osm/bbbike/)  
[Choose borders](https://extract.bbbike.org/)  

After file is downloaded, you can choose any bbox inside downloaded area:  
(bbox is set in format min_lon, min_lat, max_lon, max_lat)

In [ ]:
from visibility_graph import VisibilityGraph

filename = "../maps/kozlovo.osm.pbf"
bbox = [36.0, 56.45, 36.1, 56.5]
map_data = VisibilityGraph()
map_data.compute_geometry(filename=filename, bbox=bbox)

Data inside this area can be processed using VisibilityGraph with chosen parameters:
* Ramer-Douglas-Peucker algorithm parameters for polygons and linestrings
* bbox_comp parameter to get rid of small polygons

In [ ]:
map_data.build_dataframe(epsilon_polygon=0.003,
                         epsilon_linestring=0.001,
                         bbox_comp=10)

note: compute_geometry method requires geopandas to be installed, which might be problematic on Windows

Computed data can be saved in .h5 file to skip data processing the other time:

In [ ]:
map_data.save_geometry("../maps/kozlovo_36_5645_361_565.h5")

TODO: [OSM API](https://wiki.openstreetmap.org/wiki/Overpass_API) and [networkx](https://osmnx.readthedocs.io/en/stable/) may help download maps dynamically

# Using precomputed data and building visibility graph

In [ ]:
from visibility_graph import VisibilityGraph

map_data = VisibilityGraph()
map_data.load_geometry("../maps/kozlovo_36_5645_361_565.h5")

Visibility graph can be built and (optionally) saved as networkx graph and (optionally) visualised using mplleaflet
* inside_percent: float parameter setting the probability of an inner edge to be added (from 0 to 1)  
* graph: bool parameter indicating whether to build a networkx graph  
* None or iterable of 2 elements: colors to plot visibility graph  
    0 element: color to plot polygons  
    1 element: dict of 3 elements: colors to plot edges  
        0: edges between objects
        1: edges inside polygon
        2: road edges

In [2]:
%%time
# import mplleaflet

map_plot=('r', {0: "royalblue", 1: "r", 2: "k"})
G, fig = map_data.build_graph(inside_percent=0,
                              graph=True,
                              map_plot=None)

print('edges: ', G.number_of_edges())
print('nodes: ', G.number_of_nodes())
# mplleaflet.display(fig=fig)

edges:  2852
nodes:  286
Wall time: 32.5 s


note: using mplleaflet may be problematic on Windows

VisibilityGraph may alse be used to find incident edges for a single point  
this feature will be used for pathfinding without graph building

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet

start = ([36.35, 56.57], None, None, None, None)
incidents = map_data.incident_vertices(start)

fig = plt.figure()
plt.scatter(start[0][0], start[0][1], color='r')
for p in incidents:
    plt.scatter(p[0][0], p[0][1], color='b')
mplleaflet.display(fig=fig)

TODO:  
algorithm documentation, license  
fix rare crosses  
fix roads  